### Imports

In [ ]:
import pandas as pd
import json 
from langchain_core.prompts import PromptTemplate
import csv
import os
import re
from langchain_ollama.llms import OllamaLLM
import logging
import sys
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferWindowMemory,ConversationBufferMemory
from langchain_core.messages import SystemMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
)

logger = logging.getLogger(__name__)
logger.info("message")



message


### LLM


In [ ]:


llm = OllamaLLM(model='mistral:7b')


### Global variables

In [2]:
param_df = pd.read_csv(r"C:\Users\LENOVO\Desktop\ML_RAG\ML_Automatisation\param.csv")

users_state = r"C:\Users\LENOVO\Desktop\ML_RAG\ML_Automatisation\users_state.csv"
users_state_pred = r"C:\Users\LENOVO\Desktop\ML_RAG\ML_Automatisation\users_state_pred.csv"
users_state_seg = r"C:\Users\LENOVO\Desktop\ML_RAG\ML_Automatisation\users_state_seg.csv"
users_state_mon = r"C:\Users\LENOVO\Desktop\ML_RAG\ML_Automatisation\users_state_mon.csv"
users_state_vis = r"C:\Users\LENOVO\Desktop\ML_RAG\ML_Automatisation\users_state_vis.csv"

csv_dict = {
    "prediction": users_state_pred,
    "forecasting": users_state_pred,
    "segmentation": users_state_seg,
    "monitoring": users_state_mon,
    "visualisation": users_state_vis,
    "processing": users_state_vis,
}



memory_dict = {}


### Tools

In [3]:
def check_validity (intent,name_project:str="null",name_table:str="null",feature_to_predict:str="null",clusters:str="null",description:str="null") :
    "checks the validity of the provided informations"
    the_name_project = name_project.lower()
    pd_status = "valid" if the_name_project in param_df["project_dir"].values else "invalid"
    
    the_name_table = name_table.lower()
    dsp_status = "valid" if the_name_table in param_df["dataset_path"].values else "invalid"
    
    the_feature_to_predict = feature_to_predict.lower()
    ftp_status = "valid" if the_feature_to_predict in param_df["feature_to_predict"].values else "invalid"
    
    if intent == "prediction" or intent =="forecasting":
        return f'{{"project_dir" : "{the_name_project}", \n "pd_status": "{pd_status}",\n "dataset_path" : "{the_name_table}", \n "dsp_status":"{dsp_status}", \n "feature_to_predict": "{the_feature_to_predict}", \n "ftp_status":"{ftp_status}"}}'
    elif intent == "monitoring" :
        return f'{{"project_dir" : "{the_name_project}", \n "pd_status": "{pd_status}",\n "dataset_path" : "{the_name_table}", \n "dsp_status":"{dsp_status}"}}'
    elif intent == "segmentation":
        return f'{{"project_dir" : "{the_name_project}", \n "pd_status": "{pd_status}",\n "dataset_path" : "{the_name_table}", \n "dsp_status":"{dsp_status}", \n "clusters":"{clusters}"}}'

    elif intent == "visualisation" or intent=="processing":
        return f'{{"project_dir" : "{the_name_project}", \n "pd_status": "{pd_status}",\n "dataset_path" : "{the_name_table}", \n "dsp_status":"{dsp_status}", \n "description":"{description}"}}'


### User history


In [4]:
def get_state(user_id,intent) -> tuple:
    """
    Returns a tuple (previous_state, current_state) for a given user_id.
    If no current_state row exists, it creates one with empty fields and returns it.
    """
    previous_state = None
    current_state = None
    rows = []
    csv_file = csv_dict[intent]
    
    with open(csv_file, mode='r', newline='', encoding='utf-8') as file:
        reader = csv.DictReader(file)
        fieldnames = reader.fieldnames
        rows = list(reader)

        for row in rows:
            if row['user_id'] == user_id:
                if row.get("status_action") == "previous_state":
                    previous_state = dict(row)
                elif row.get("status_action") == "current_state" or row.get("status_action") == "":
                    current_state = dict(row)

    # If current_state is not found, create it with empty fields and save
    if not current_state:
        current_state = {field: "" for field in fieldnames}
        current_state["user_id"] = user_id
        current_state["status_action"] = "current_state"
        current_state["status"] = "ongoing"
        current_state["type_action"] = ""

        rows.append(current_state)

        # Save the updated rows with the new current_state
        with open(csv_file, mode='w', newline='', encoding='utf-8') as file:
            writer = csv.DictWriter(file, fieldnames=fieldnames)
            writer.writeheader()
            writer.writerows(rows)

    return previous_state, current_state



### Parse JSON

In [5]:

def clean_and_parse(json_str):
    try:
        # Find all JSON-like blocks (starts with { and ends with })
        matches = re.findall(r'\{.*?\}', json_str.strip(), re.DOTALL)

        if not matches:
            raise ValueError("No valid JSON object found.")

        for match in matches:
            try:
                # Remove trailing commas (common LLM mistake)
                cleaned = re.sub(r',(\s*[}\]])', r'\1', match)
                return json.loads(cleaned)
            except json.JSONDecodeError:
                continue  # Try next match if this one fails

        raise ValueError("All extracted JSON blocks failed to parse.")

    except Exception as e:
        print(f"Please reformulate your prompt : {e}")
        return None


### New State (replace valid fields from message with previous user state)

In [6]:
def new_user_state(user_row, infos_agent,intent):
    
    """
    Updates user_row dict with values from validate_data dict for prediction intent.
    - For "dsp_status", "pd_status", "ftp_status":
        - If status is "valid", update both the status and its related value in user_row from validate_data.
        - If status is "invalid", leave both fields unchanged.
    - Leave other fields unchanged.
    Returns the updated user_row as a valid JSON string.
    """

    # Make a copy to avoid mutating the original
    validate_data = clean_and_parse(infos_agent)
    updated_row = user_row.copy() 
    # Update dataset_path and dsp_status
    if "dsp_status" in validate_data and validate_data["dsp_status"] == "valid":
        updated_row["dsp_status"] = "valid"
        if "dataset_path" in validate_data:
            updated_row["dataset_path"] = validate_data["dataset_path"]

    # Update project_dir and pd_status
    if "pd_status" in validate_data and validate_data["pd_status"] == "valid":
        updated_row["pd_status"] = "valid"
        if "project_dir" in validate_data:
            updated_row["project_dir"] = validate_data["project_dir"]

    # Update feature_to_predict and ftp_status
    if intent in ["prediction","forecasting"] :
        if "ftp_status" in validate_data and validate_data["ftp_status"] == "valid":
            updated_row["ftp_status"] = "valid"
            if "feature_to_predict" in validate_data:
                updated_row["feature_to_predict"] = validate_data["feature_to_predict"]
    if intent == "segmentation" :
        clusters_value = validate_data.get("clusters")
        if clusters_value not in [None, "not found", "null"]:
            updated_row["clusters"] = clusters_value # This will be False for None, "", 0, [], etc.
            
    if intent in ["visualisation", "processing"]:
        desc_value = validate_data.get("description")
        current_desc = updated_row.get("description", "").strip()
            # Add a separator if there's already something written
        if current_desc:
            updated_row["description"] = current_desc + " | " + desc_value.strip()
        else:
            updated_row["description"] = desc_value.strip()

    return updated_row




### Agent : Prediction & Tendance

In [7]:

def infos_validation_pred(intent,query,previous_state) :
    prompt = PromptTemplate.from_template("""
You are a helpful assistant for a prediction task. Your job has two parts:

---

**Step 1 – Extract three specific fields from the user's input**:  
These fields are:
- "project_dir" (project directory name)  
- "dataset" (CSV or dataset name)  
- "feature_to_predict" (the prediction target)  

For **each field**, follow this exact logic:

1. If the user **explicitly provides a value** for this field → use that value.  
2. Else if the user **explicitly asks to keep or reuse the previous value** for this field (e.g., "keep the same dataset", "use previous project_dir") → set it to `"previous"`.  
3. Else → set it to `"not found"`.

⚠️ You must **evaluate each field separately**.  
⚠️ Do **NOT** assume or guess that a field should be `"previous"` unless the user says so **clearly and specifically for that exact field**.  
⚠️ If the user mentions only one field, all other fields must be set to `"not found"` unless also mentioned.

---

**Step 2 – Respond ONLY with JSON, no extra text**:  
Format:  
{{
  "project_dir": "<value>" or "previous" or "not found",
  "dataset": "<value>" or "previous" or "not found",
  "feature_to_predict": "<value>" or "previous" or "not found"
}}

User input: {input}
""")



            
    response = llm.invoke(prompt.format(input=query))
    print("LLM reponse ",response)
    clean_reponse = clean_and_parse(response)
    
    if not clean_reponse:
        return None
    if previous_state : 
        if clean_reponse["project_dir"] == "previous" :
            clean_reponse["project_dir"] = previous_state["project_dir"]
        if clean_reponse["dataset"] == "previous" :
            clean_reponse["dataset"] = previous_state["dataset_path"]
        if clean_reponse["feature_to_predict"] == "previous" :
            clean_reponse["feature_to_predict"] = previous_state["feature_to_predict"]    
        
    print("Cleaned response:", clean_reponse)
    infos_validity = check_validity(intent=intent,name_project=clean_reponse["project_dir"],name_table=clean_reponse["dataset"],feature_to_predict=clean_reponse["feature_to_predict"])
    return(infos_validity)
    


### Agent : Segmentation

In [8]:
def infos_validation_seg(intent,query,previous_state) :
    
  prompt = PromptTemplate.from_template("""
You are a helpful assistant for a segmentation task. Your job has two parts:

---

**Step 1 – Extract three specific fields from the user's input**:  
These fields are:
- "project_dir" (project directory name)  
- "dataset" (CSV or dataset name)  
- "Clusters" (the clusters number)  

For **each field**, follow this exact logic:

1. If the user **explicitly provides a value** for this field → use that value.  
2. Else if the user **explicitly asks to keep or reuse the previous value** for this field (e.g., "keep the same dataset", "use previous project_dir") → set it to `"previous"`.  
3. Else → set it to `"not found"`.

⚠️ You must **evaluate each field separately**.  
⚠️ Do **NOT** assume or guess that a field should be `"previous"` unless the user says so **clearly and specifically for that exact field**.  
⚠️ If the user mentions only one field, all other fields must be set to `"not found"` unless also mentioned.

---

**Step 2 – Respond ONLY with JSON, no extra text**:  
Format:  
{{
  "project_dir": "<value>" or "previous" or "not found",
  "dataset": "<value>" or "previous" or "not found",
  "clusters": "<value>" or "previous" or "not found"
}}

User input: {input}
""")

  response = llm.invoke(prompt.format(input=query))
  print("LLM reponse ",response)
  clean_reponse = clean_and_parse(response)
  if previous_state : 
        if clean_reponse["project_dir"] == "previous" :
            clean_reponse["project_dir"] = previous_state["project_dir"]
        if clean_reponse["dataset"] == "previous" :
            clean_reponse["dataset"] = previous_state["dataset_path"]
        if clean_reponse["clusters"] == "previous" :
            clean_reponse["clusters"] = previous_state["clusters"]    
  infos_validity = check_validity(intent,name_project=clean_reponse["project_dir"],name_table=clean_reponse["dataset"],clusters=clean_reponse["clusters"])
  return(infos_validity)
    

### Agent : monitoring

In [9]:
def infos_validation_mon(intent,query,previous_state) :
    
    prompt = PromptTemplate.from_template("""
You are a helpful assistant for a monitoring task. Your job has two parts:

---

**Step 1 – Extract two specific fields from the user's input**:  
These fields are:
- "project_dir" (project directory name)  
- "dataset" (CSV or dataset name)  

For **each field**, follow this exact logic:

1. If the user **explicitly provides a value** for this field → use that value.  
2. Else if the user **explicitly asks to keep or reuse the previous value** for this field (e.g., "keep the same dataset", "use previous project_dir") → set it to `"previous"`.  
3. Else → set it to `"not found"`.

⚠️ You must **evaluate each field separately**.  
⚠️ Do **NOT** assume or guess that a field should be `"previous"` unless the user says so **clearly and specifically for that exact field**.  
⚠️ If the user mentions only one field, all other fields must be set to `"not found"` unless also mentioned.

---

**Step 2 – Respond ONLY with JSON, no extra text**:  
Format:  
{{
  "project_dir": "<value>" or "previous" or "not found",
  "dataset": "<value>" or "previous" or "not found"
}}

User input: {input}
""")

    response = llm.invoke(prompt.format(input=query))
    print("LLM reponse ",response)
    clean_reponse = clean_and_parse(response)
    if previous_state : 
        if clean_reponse["project_dir"] == "previous" :
            clean_reponse["project_dir"] = previous_state["project_dir"]
        if clean_reponse["dataset"] == "previous" :
            clean_reponse["dataset"] = previous_state["dataset_path"]
 
    infos_validity = check_validity(intent,name_project=clean_reponse["project_dir"],name_table=clean_reponse["dataset"])
    return(infos_validity)
    

### Visualisation and Processing

In [31]:
import logging

def infos_validation_vis_pro(intent, query, previous_state):
    """
    Extracts and validates project_dir and dataset for visualization/processing tasks.
    """
    prompt = PromptTemplate.from_template(f"""
You are a helpful assistant for a {intent} task. Your job has two parts:

---

**Step 1 – Extract two specific fields from the user's input**:
These fields are:
- "project_dir" (project directory name)
- "dataset" (CSV or dataset name)

For each field, follow exactly:
1. If the user explicitly provides a value → use that value.
2. Else if the user explicitly asks to keep/reuse the previous value → set it to "previous".
3. Else → set it to "not found".

⚠️ Evaluate each field separately.
⚠️ Do NOT assume or guess "previous" unless explicitly stated.
⚠️ If the user mentions only one field, set all others to "not found".

---

**Step 2 – Respond ONLY with JSON, no extra text**:
Format:
{{
  "project_dir": "<value>" or "previous" or "not found",
  "dataset": "<value>" or "previous" or "not found"
}}

User input: {query}
""")

    try:
        response = llm.invoke(prompt.format(intent=intent,query=query))
        logging.debug("LLM raw response: %s", response)
        
        clean_response = clean_and_parse(response)

        # Vérification des clés
        for key in ["project_dir", "dataset"]:
            if key not in clean_response:
                logging.error("Missing key '%s' in LLM response", key)
                clean_response[key] = "not found"

        # Gestion des valeurs "previous"
        if previous_state:
            if clean_response["project_dir"] == "previous":
                clean_response["project_dir"] = previous_state.get("project_dir", "not found")
            if clean_response["dataset"] == "previous":
                clean_response["dataset"] = previous_state.get("dataset_path", "not found")

        clean_response["description"] = query

        infos_validity = check_validity(
            intent,
            name_project=clean_response["project_dir"],
            name_table=clean_response["dataset"],
            description=clean_response["description"]
        )
        return infos_validity

    except Exception as e:
        logging.exception("Error during infos_validation_vis_pro execution: %s", e)
        return {"error": str(e), "status": "failed"}


### Execution Agents

In [29]:
def agent_execution(valid_fields, intent):
  
    valid_intents = {
        "prediction",
        "forecasting",
        "segmentation",
        "monitoring",
        "visualisation",
        "processing"
    }
    
    if intent in valid_intents:
        return f"✅ The {intent} Agent is being executed !!!"
    
    return f"❌ Unknown intent: {intent}. Execution aborted."


### Diplay valid/invalid fields

In [18]:
import logging

logger = logging.getLogger(__name__)

def validate_data1(valid_fields1, invalid_fields):
    """Logs the result of a validation check on provided fields."""
    logger.info("-" * 40)
    logger.info("Validation Results:")

    if valid_fields1:
        logger.info("✅ Valid fields : ")
        max_key_len = max(len(k) for k in valid_fields1)
        for key, value in valid_fields1.items():
            if key == "Description":
                continue
            logger.info(f"{key.ljust(max_key_len)} : {value}")
    else:
        logger.info("⚠️ No valid fields provided.")

    if invalid_fields:
        logger.warning("❌ Invalid fields: \n %s", ", ".join(invalid_fields))
    else:
        logger.info("✅ No invalid fields detected.")

    logger.info("-" * 40)
    logger.info("ℹ️ Awaiting missing or corrected information.")

def validate_data2(user_input, valid_fields1, intent):
    """
    Validates the user's confirmation based on input.
    Returns 'yes' / 'no' / None depending on user confirmation.
    """
    try:
        prompt = PromptTemplate.from_template("""
You are a confirmation-checking assistant. Your task is to decide if the user is explicitly confirming an action.

Rules:
- Respond only with **yes** if the user clearly and explicitly confirms using words like "yes", "confirm", "okay", "proceed", or "go ahead".
- If the user is still providing information, asking questions, making changes, or specifying details (e.g., project names, paths, numbers, options), respond with **no**.
- Do not infer confirmation from intent; only explicit confirmation counts.
- Respond with exactly "yes" or "no". Nothing else.

User Input: {user_input}

Examples:
- "I want to use 4 clusters" → no
- "I want to use project_1 directory" → no
- "Yes, I confirm" → yes
- "Please proceed" → yes
- "Use project_2" → no
""")
        response = llm.invoke(prompt.format(user_input=user_input))
        cleaned_response = response.strip().lower()

        if cleaned_response == "yes":
            return "yes"
        else:
            logger.info("-" * 40)
            logger.info("✅ All provided information is valid:")
            max_key_len = max(len(k) for k in valid_fields1)
            for key, value in valid_fields1.items():
                logger.info(f"{key.ljust(max_key_len)} : {value}")
            logger.info("-" * 40)
            logger.info(f"❓ Awaiting confirmation to execute the '{intent}' agent.")
            return None

    except Exception as e:
        logger.exception(f"Error during confirmation prompt: {e}")
        raise


### General 


In [25]:

def llm_general(user_id, query):
    if not query:
        return "⚠️ No query provided."

    try:
        if user_id not in memory_dict:
            memory_dict[user_id] = ConversationBufferMemory(
                memory_key="chat_history",
                return_messages=True,
                input_key="text"
            )
        memory = memory_dict[user_id]

        prompt = ChatPromptTemplate(
            [
                SystemMessage(content="""You are an AI expert that only handles AI-related questions, and also assist users about task executions for our machine learning automation purpose, in the following categories:

1. Prediction/Forecasting Task
2. Segmentation Task
3. Monitoring Task
4. Data Processing/Visualization Task

Instructions:
- If the user input is not related to AI or the tasks above, respond with:
  **"Sorry, we don't handle such type of question, please enter a question related to AI."**

- If the user wants to perform a task, assist them with the required information:
    • For **Prediction/Forecasting**: requires `project directory name`, `dataset name`, and `target feature to predict`.
    • For **Segmentation**: requires `project directory name`, `dataset name`, and `number of clusters`.
    • For **Monitoring**: requires `project directory name` and `dataset name`.
    • For **Data Processing/Visualization**: requires `project directory name`, `dataset name`, and a clear `description of what the agent should do`.

-if the user asks a general question related to AI:
    • 1. Be **Clear and Concise**: Avoid unnecessary jargon, and break down complex concepts into easy-to-understand explanations. Aim for clarity while maintaining technical accuracy.
    • 2. Provide **Examples**: Use practical examples to illustrate concepts, making them relatable and easier to grasp.


User Input: {user_input}

"""),
                MessagesPlaceholder(variable_name="chat_history"),
                HumanMessagePromptTemplate.from_template("{text}"),
            ]
        )

        general_out = LLMChain(llm=llm, prompt=prompt, memory=memory)
        response = general_out.invoke({"text": query})

        return response.get("text", "⚠️ No valid response from LLM.")

    except Exception as e:
        # Logging en prod préférable à un print
        print(f"❌ Error in llm_general: {e}")
        return "⚠️ An error occurred while processing your request."

    

### Clear user history (for switched/complete path)

In [15]:
def clear_current_state(user_id):
    csv_path = users_state
    rows = []
    with open(csv_path, mode='r', newline='', encoding='utf-8') as file:
        reader = csv.DictReader(file)
        fieldnames = reader.fieldnames
        for row in reader:
            if row['user_id'] == str(user_id) :
                if row['status_action'] != "previous_state" :
                    for key in row:
                        if key != "user_id":
                            row[key] = ""
                
            rows.append(row)
    with open(csv_path, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.DictWriter(file, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(rows)
    print(f"Cleared user state for user_id={user_id}")


### Insert row 

In [26]:
import csv
import logging

logger = logging.getLogger(__name__)

def update_row(user_input, update_dict, intent):
    """
    Updates a user's row in the CSV based on intent and update_dict.
    Keeps previous state if current is not validated; removes it if validated.
    Returns valid fields and a list of invalid fields.
    """

    try:
        csv_path = csv_dict[intent]
        user_id = str(update_dict.get("user_id"))
        rows = []
        invalid_fields = []
        validated_fields = {}
        updated = False
        previous_state = None
        row_to_update = None
        response = None  

        with open(csv_path, mode='r', newline='', encoding='utf-8') as file:
            reader = csv.DictReader(file)
            fieldnames = reader.fieldnames

            for row in reader:
                if row['user_id'] == user_id:
                    if row['status_action'] != "previous_state":
                        for key, value in update_dict.items():
                            if key in row:
                                row[key] = value

                            if key == "dsp_status":
                                if value != "valid":
                                    invalid_fields.append("Dataset path")
                                else:
                                    validated_fields["Dataset path"] = row["dataset_path"]

                            if key == "pd_status":
                                if value != "valid":
                                    invalid_fields.append("Project directory")
                                else:
                                    validated_fields["Project directory"] = row["project_dir"]

                            if intent == "segmentation":
                                if key == "clusters" and not value:
                                    invalid_fields.append("Clusters")
                                elif key == "clusters" and value:
                                    validated_fields["Clusters"] = value

                            if intent in ("prediction", "forecasting"):
                                if key == "ftp_status":
                                    if value != "valid":
                                        invalid_fields.append("Feature to predict")
                                    else:
                                        validated_fields["Feature to predict"] = row["feature_to_predict"]

                            if intent in ("visualisation", "processing"):
                                if key == "description":
                                    validated_fields["Description"] = value
                                    desc = value

                        row["type_action"] = intent

                        if invalid_fields:
                            validate_data1(validated_fields, invalid_fields)
                        else:
                            if intent in ("visualisation", "processing"):
                                prompt = PromptTemplate.from_template("""You are given an unstructured string containing information about a data {intent} task. This string includes the dataset name, the project directory, and a description (details) of the task.

Your job is to extract and rephrase the description of the task into a clear, complete sentence suitable for an automated agent to execute.

The structured sentence should follow this exact format:

"the task is to `<task_description>`."

### Input:
{desc}

### Output:
A single clear sentence following the format above.""")

                                summary = llm.invoke(prompt.format(intent=intent, desc=validated_fields["Description"]))
                                logger.info(f"Summary : {summary}")
                                validated_fields["Description"] = summary
                                row["description"] = summary

                            response = validate_data2(user_input, validated_fields, intent)

                        if response:
                            if response == "yes":
                                result = agent_execution(validated_fields, intent)
                                logger.info(f"Agent Output: {result}")
                                row["status"] = "validated"
                            else:
                                row["status"] = "ongoing"
                        else:
                            row["status"] = "ongoing"

                        row["status_action"] = "previous_state" if row["status"] == "validated" else "current_state"
                        updated = True
                        row_to_update = row

                    elif row['status_action'] == "previous_state":
                        previous_state = row
                else:
                    rows.append(row)

        if not updated:
            raise ValueError("No editable row found for this user_id.")

        if row_to_update:
            if row_to_update["status"] == "validated":
                rows.append(row_to_update)
            else:
                if previous_state:
                    rows.append(previous_state)
                rows.append(row_to_update)

        with open(csv_path, mode='w', newline='', encoding='utf-8') as file:
            writer = csv.DictWriter(file, fieldnames=fieldnames)
            writer.writeheader()
            writer.writerows(rows)

        return validated_fields, invalid_fields

    except Exception as e:
        logger.exception(f"❌ Failed to update row for user_id={update_dict.get('user_id')}: {e}")
        raise


### Test

In [27]:

logging.basicConfig(
    level=logging.INFO,
    format="%(message)s",
    handlers=[logging.StreamHandler(sys.stdout)]
)

logger = logging.getLogger(__name__)


validation_funcs = {
    "segmentation": infos_validation_seg,
    "prediction": infos_validation_pred,
    "forecasting": infos_validation_pred,
    "monitoring": infos_validation_mon,
    "visualisation": infos_validation_vis_pro,
    "processing": infos_validation_vis_pro,
}


def agent_manager1(user_id, intent, user_input):
    try:
        if intent != "general":
            previous_state, current_state = get_state(user_id, intent)
            type_action = current_state.get("type_action", None)
            logger.info(f"Current: {current_state}, Intent: {intent}, Input: {user_input}")

            if intent in validation_funcs:
                infos = validation_funcs[intent](intent, user_input, previous_state)
                if not infos:
                    raise ValueError("Validation failed.")
                user_state = new_user_state(current_state, infos, intent)
                valid_fields, invalid_fields = update_row(user_input, user_state, intent)
            else:
                logger.warning("Intent not handled")
        else:
            out = llm_general(user_id=user_id, query=user_input)
            logger.info(out)

    except Exception as e:
        logger.exception(f"Agent manager failed: {e}")


In [32]:
user_id = "7" 
user_input = "i want use table 'housing.csv'"
intent = "visualisation"  
agent_manager1(user_id, intent, user_input) 

Current: {'user_id': '7', 'dataset_path': 'housing.csv', 'dsp_status': 'valid', 'project_dir': 'project_1', 'pd_status': 'valid', 'description': ' "The task is to visualize data from the \'housing.csv\' dataset, using either \'table_1.csv\' or the previously used table."', 'type_action': 'visualisation', 'status_action': 'current_state', 'status': 'ongoing'}, Intent: visualisation, Input: i want use table 'housing.csv'


HTTP Request: POST http://127.0.0.1:11434/api/generate "HTTP/1.1 200 OK"
LLM reponse   {
  "project_dir": "not found",
  "dataset": "housing.csv"
}
HTTP Request: POST http://127.0.0.1:11434/api/generate "HTTP/1.1 200 OK"
Summary :  "The task is to visualize data from the 'housing.csv' dataset using either 'table_1.csv' or the previously used table."
HTTP Request: POST http://127.0.0.1:11434/api/generate "HTTP/1.1 200 OK"
----------------------------------------
✅ All provided information is valid:
Dataset path      : housing.csv
Project directory : project_1
Description       :  "The task is to visualize data from the 'housing.csv' dataset using either 'table_1.csv' or the previously used table."
----------------------------------------
❓ Awaiting confirmation to execute the 'visualisation' agent.
